In [129]:
# import json as json
# import pandas as pd
# import requests as requests
# def getLatLng(address):
#     url = 'https://dapi.kakao.com/v2/local/search/address.json?query=' + address
#     headers = {"Authorization": "KakaoAK 073a393656181c6073880062d3507191"}
#     result = json.loads(str(requests.get(url, headers=headers).text))
#     match_first = result['documents'][0]['address']
#     return float(match_first['y']), float(match_first['x'])
# getLatLng('부산 사직동')

In [5]:
import requests; from urllib.parse import urlparse
import pandas as pd
import geopandas
# 필요한 module다운받기 -> geopandas가 import되지 않는다면? 직접 anaconda prompt에서 설치해야한다
# conda install -c conda-forge geopandas (근데 설치시간이 드릅게 오래 걸린다!)

In [128]:
address = "강원 원주시 단구로 171"
url = "https://dapi.kakao.com/v2/local/search/address.json?query=" + address
# 카카오맵 주소검색 url에다 address를 붙여 요청
result = requests.get(urlparse(url).geturl(),
                      headers={"Authorization": "KakaoAK 073a393656181c6073880062d3507191"})
json_obj = result.json()
print(json_obj)
# 요청시 나온 결과(result)를 json으로 담아서 print해본 것.
# json으로 담는게 (json_obj = result.json() <- 이 행위) 뭘까????? 
# 그냥 result만 print하면 <Response [200]> 가 뜨고 만다. 띠요용
# 카카오맵 document에 있는 녀석들이 몽땅 나온다. 얘네들은 각 index안에 값이 담긴 list형태다

{'documents': [{'address': {'address_name': '강원 원주시 명륜동 242-2', 'b_code': '4213010600', 'h_code': '4213054100', 'main_address_no': '242', 'mountain_yn': 'N', 'region_1depth_name': '강원', 'region_2depth_name': '원주시', 'region_3depth_h_name': '명륜1동', 'region_3depth_name': '명륜동', 'sub_address_no': '2', 'x': '127.94774501944', 'y': '37.3369873534915'}, 'address_name': '강원 원주시 단구로 171', 'address_type': 'ROAD_ADDR', 'road_address': {'address_name': '강원 원주시 단구로 171', 'building_name': '국립공원연구원', 'main_building_no': '171', 'region_1depth_name': '강원', 'region_2depth_name': '원주시', 'region_3depth_name': '명륜동', 'road_name': '단구로', 'sub_building_no': '', 'underground_yn': 'N', 'x': '127.94774501944', 'y': '37.3369873534915', 'zone_no': '26441'}, 'x': '127.94774501944', 'y': '37.3369873534915'}], 'meta': {'is_end': True, 'pageable_count': 1, 'total_count': 1}}


In [130]:
list = []
for document in json_obj['documents']:
    val = [document['road_address']['building_name'], document['address_name'], document['y'], document['x']]
    list.append(val)
df = pd.DataFrame(list, columns=['building_name', 'address_name','lat','lon'])
df
# 나온 결과들 중 필요한 녀석들('address_name','x','y')을 잡아 list형태로 만들어 DataFrame안에 넣어주자!

,building_name,address_name,lat,lon
0,국립공원연구원,강원 원주시 단구로 171,37.3369873534915,127.94774501944


In [136]:
df = pd.read_excel('주소 시험용.xlsx', sheet_name='Sheet1')
df 

,주소
0,서울 강남구 선릉로 836 삼원빌딩 1~2층
1,서울 동작구 보라매로5길 43 삼성쉐르빌 1층
2,서울 영등포구 여의대방로65길 24 호성빌딩1층


In [134]:
def address_to_latlon(address):
    url = "https://dapi.kakao.com/v2/local/search/address.json?query=" + address
    result = requests.get(urlparse(url).geturl(),
                      headers={"Authorization": "KakaoAK 073a393656181c6073880062d3507191"})
    json_obj = result.json()
    for document in json_obj['documents']:
        val = [document['address_name'], document['y'],document['x']]
    return val

#위의 두 녀석들을 합쳐 address가 입력되면 address_name, y, x를 반환하는 함수로 만들자.

In [132]:
address_to_latlon("서울 동작구 보라매로5길 43")
# 성공! 이면 정말 좋겠지만 .............

['서울 동작구 보라매로5길 43', '37.4914310798968', '126.923417000996']

In [137]:
list = []
for address in df['주소']:
    list.append(address_to_latlon(address))
    
df1 = pd.DataFrame(list, columns = ['address_name', 'lat', 'lon'])
df1.head()
# 받아온 엑셀파일(df) 속 '주소' column들을 for문으로 함수에 쏙쏙.

,address_name,lat,lon
0,서울 강남구 선릉로 836,37.5265428105713,127.040533570867
1,서울 동작구 보라매로5길 43,37.4914310798968,126.923417000996
2,서울 영등포구 여의대방로65길 24,37.5204421498979,126.929293717613
